In [ ]:
!pip install xgboost
!pip install lightgbm
!pip install catboost

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import classification_report
import warnings

In [3]:
warnings.filterwarnings('ignore')

In [ ]:
!pip uninstall -y numpy pandas lightgbm xgboost catboost dask xarray
!pip install pandas==2.2.0
!pip install lightgbm xgboost catboost ppscore

In [ ]:
!pip install -U pandas==2.2.2

In [ ]:
import pandas as pd
import ppscore as pps

pd.__version__

In [22]:
df=pd.read_csv('/content/Lead Scoring.csv')
df

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,...,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.00,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.50,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.00,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.00,...,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.00,...,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,19d6451e-fcd6-407c-b83b-48e1af805ea9,579564,Landing Page Submission,Direct Traffic,Yes,No,1,8.0,1845,2.67,...,No,Potential Lead,Mumbai,02.Medium,01.High,15.0,17.0,No,No,Email Marked Spam
9236,82a7005b-7196-4d56-95ce-a79f937a158d,579546,Landing Page Submission,Direct Traffic,No,No,0,2.0,238,2.00,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,19.0,No,Yes,SMS Sent
9237,aac550fe-a586-452d-8d3c-f1b62c94e02c,579545,Landing Page Submission,Direct Traffic,Yes,No,0,2.0,199,2.00,...,No,Potential Lead,Mumbai,02.Medium,01.High,13.0,20.0,No,Yes,SMS Sent
9238,5330a7d1-2f2b-4df4-85d6-64ca2f6b95b9,579538,Landing Page Submission,Google,No,No,1,3.0,499,3.00,...,No,NaN,Other Metro Cities,02.Medium,02.Medium,15.0,16.0,No,No,SMS Sent


In [23]:
df=df.drop(columns=['Prospect ID','Lead Number',])
df.columns

Index(['Lead Origin', 'Lead Source', 'Do Not Email', 'Do Not Call',
       'Converted', 'TotalVisits', 'Total Time Spent on Website',
       'Page Views Per Visit', 'Last Activity', 'Country', 'Specialization',
       'How did you hear about X Education', 'What is your current occupation',
       'What matters most to you in choosing a course', 'Search', 'Magazine',
       'Newspaper Article', 'X Education Forums', 'Newspaper',
       'Digital Advertisement', 'Through Recommendations',
       'Receive More Updates About Our Courses', 'Tags', 'Lead Quality',
       'Update me on Supply Chain Content', 'Get updates on DM Content',
       'Lead Profile', 'City', 'Asymmetrique Activity Index',
       'Asymmetrique Profile Index', 'Asymmetrique Activity Score',
       'Asymmetrique Profile Score',
       'I agree to pay the amount through cheque',
       'A free copy of Mastering The Interview', 'Last Notable Activity'],
      dtype='object')

In [24]:
df.isnull().sum()

,0
Lead Origin,0
Lead Source,36
Do Not Email,0
Do Not Call,0
Converted,0
TotalVisits,137
Total Time Spent on Website,0
Page Views Per Visit,137
Last Activity,103
Country,2461


In [25]:
cat_cols = df.select_dtypes(include='object').columns
cat_cols.str.lower()

Index(['lead origin', 'lead source', 'do not email', 'do not call',
       'last activity', 'country', 'specialization',
       'how did you hear about x education', 'what is your current occupation',
       'what matters most to you in choosing a course', 'search', 'magazine',
       'newspaper article', 'x education forums', 'newspaper',
       'digital advertisement', 'through recommendations',
       'receive more updates about our courses', 'tags', 'lead quality',
       'update me on supply chain content', 'get updates on dm content',
       'lead profile', 'city', 'asymmetrique activity index',
       'asymmetrique profile index',
       'i agree to pay the amount through cheque',
       'a free copy of mastering the interview', 'last notable activity'],
      dtype='object')

In [26]:
for col in cat_cols:
    if df[col].isnull().sum()>0:
        df[col].fillna('missing', inplace=True)

In [27]:
df[cat_cols]=df[cat_cols].apply(lambda col:col.str.lower())
df[cat_cols]=df[cat_cols].apply(lambda col:col.str.strip())

In [28]:
num_cols=df.select_dtypes(include=['int64','float64']).columns
num_cols.str.lower()

Index(['converted', 'totalvisits', 'total time spent on website',
       'page views per visit', 'asymmetrique activity score',
       'asymmetrique profile score'],
      dtype='object')

In [29]:
for col in num_cols:
    if df[col].isnull().sum()>0:
        df.fillna(df[col].median(), inplace=True)

In [30]:
df.isnull().sum()

,0
Lead Origin,0
Lead Source,0
Do Not Email,0
Do Not Call,0
Converted,0
TotalVisits,0
Total Time Spent on Website,0
Page Views Per Visit,0
Last Activity,0
Country,0


In [31]:
pps_all = pps.predictors(df, y="Lead Quality")
pps_all.sort_values(by="ppscore", ascending=False)

,x,y,ppscore,case,is_valid_score,metric,baseline_score,model_score,model
0,Tags,Lead Quality,4.575284e-01,classification,True,weighted F1,0.365358,0.655725,DecisionTreeClassifier()
1,Lead Profile,Lead Quality,2.041730e-01,classification,True,weighted F1,0.365358,0.494935,DecisionTreeClassifier()
2,What is your current occupation,Lead Quality,1.597781e-01,classification,True,weighted F1,0.365358,0.466760,DecisionTreeClassifier()
3,Converted,Lead Quality,1.394641e-01,classification,True,weighted F1,0.365358,0.453868,DecisionTreeClassifier()
4,Last Notable Activity,Lead Quality,1.203056e-01,classification,True,weighted F1,0.365358,0.441709,DecisionTreeClassifier()
5,Lead Source,Lead Quality,7.131094e-02,classification,True,weighted F1,0.365358,0.410615,DecisionTreeClassifier()
6,Asymmetrique Profile Score,Lead Quality,6.381633e-02,classification,True,weighted F1,0.365358,0.405858,DecisionTreeClassifier()
7,Lead Origin,Lead Quality,6.119436e-02,classification,True,weighted F1,0.365358,0.404194,DecisionTreeClassifier()
8,Total Time Spent on Website,Lead Quality,2.838968e-02,classification,True,weighted F1,0.365358,0.383375,DecisionTreeClassifier()
9,Last Activity,Lead Quality,2.041883e-02,classification,True,weighted F1,0.365358,0.378317,DecisionTreeClassifier()


In [32]:
useful_features = pps_all[
    pps_all["ppscore"] > 0.05
]["x"].tolist()

df_model = df[useful_features + ["Lead Quality"]]

In [33]:
pps.score(df, x="TotalVisits", y="Lead Quality")

{'x': 'TotalVisits',
 'y': 'Lead Quality',
 'ppscore': 0.0029108637287932615,
 'case': 'classification',
 'is_valid_score': True,
 'metric': 'weighted F1',
 'baseline_score': 0.365357969118032,
 'model_score': 0.367205325586494,
 'model': DecisionTreeClassifier()}

In [34]:
df.columns = df.columns.str.lower().str.strip()

In [35]:
x=df.drop(columns=['converted'])
y=df['converted']

In [36]:
x = pd.get_dummies(x, drop_first=True)

In [37]:
k = 30
chi_selector = SelectKBest(score_func=chi2, k=k)
x_chi = chi_selector.fit_transform(x, y)

In [38]:
selected_features = x.columns[chi_selector.get_support()]
print("Selected features:\n", selected_features)

Selected features:
 Index(['total time spent on website', 'lead origin_lead add form',
       'lead source_reference', 'lead source_welingak website',
       'do not email_yes', 'last activity_email bounced',
       'last activity_olark chat conversation', 'last activity_sms sent',
       'specialization_missing', 'how did you hear about x education_missing',
       'how did you hear about x education_select',
       'what is your current occupation_missing',
       'what is your current occupation_working professional',
       'what matters most to you in choosing a course_missing',
       'tags_closed by horizzon', 'tags_interested in other courses',
       'tags_lost to eins', 'tags_missing', 'tags_ringing',
       'tags_switched off', 'tags_will revert after reading the email',
       'lead quality_low in relevance', 'lead quality_might be',
       'lead quality_missing', 'lead quality_worst', 'lead profile_missing',
       'lead profile_potential lead', 'asymmetrique activity inde

In [39]:
x_selected = x[selected_features]
x_train, x_test, y_train, y_test = train_test_split(x_selected, y, test_size=0.2, random_state=42)

In [40]:
x_train = pd.get_dummies(x_train, drop_first=True)
x_test = pd.get_dummies(x_test, drop_first=True)
x_test = x_test.reindex(columns=x_train.columns, fill_value=0)

In [41]:
models = {
    "Logistic Regression": LogisticRegression(class_weight='balanced',max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42, verbose=0),
    "SVM": SVC(random_state=42),
    "CatBoosting": CatBoostClassifier(verbose=0),
    "XGBoostingClassifier": XGBClassifier(verbosity=0),
    "LightGBM":LGBMClassifier(verbosity=0),
    "AdaBoostClassifier":AdaBoostClassifier()
}

In [42]:
trained_models = {}
for name, model in models.items():
    model.fit(x_train, y_train)
    trained_models[name] = model
    y_pred = model.predict(x_test)
    print(name)
    print(classification_report(y_test, y_pred))

Logistic Regression
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1107
           1       0.92      0.92      0.92       741

    accuracy                           0.93      1848
   macro avg       0.93      0.93      0.93      1848
weighted avg       0.93      0.93      0.93      1848

Decision Tree
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      1107
           1       0.89      0.89      0.89       741

    accuracy                           0.91      1848
   macro avg       0.91      0.91      0.91      1848
weighted avg       0.91      0.91      0.91      1848

Random Forest
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1107
           1       0.92      0.91      0.91       741

    accuracy                           0.93      1848
   macro avg       0.93      0.93      0.93      1848
weighted avg       0.93   

In [45]:
import joblib
joblib.dump(selected_features, "chi_selected_features.pkl")

['chi_selected_features.pkl']

In [49]:
model = joblib.load("/content/lead_model.pkl")
train_columns = joblib.load("/content/train_columns.pkl")

In [50]:
new_df = pd.read_csv("/content/new_leads_test.csv")
new_df.columns

Index(['prospect id', 'lead number', 'lead origin', 'lead source',
       'do not email', 'do not call', 'converted', 'totalvisits',
       'total time spent on website', 'page views per visit', 'last activity',
       'country', 'specialization', 'how did you hear about x education',
       'what is your current occupation',
       'what matters most to you in choosing a course', 'search', 'magazine',
       'newspaper article', 'x education forums', 'newspaper',
       'digital advertisement', 'through recommendations',
       'receive more updates about our courses', 'tags', 'lead quality',
       'update me on supply chain content', 'get updates on dm content',
       'lead profile', 'city', 'asymmetrique activity index',
       'asymmetrique profile index', 'asymmetrique activity score',
       'asymmetrique profile score',
       'i agree to pay the amount through cheque',
       'a free copy of mastering the interview', 'last notable activity'],
      dtype='object')

In [51]:
cat_cols_new = new_df.select_dtypes(include='object').columns
cat_cols_new.str.lower()

Index(['prospect id', 'lead origin', 'lead source', 'do not email',
       'do not call', 'last activity', 'country', 'specialization',
       'how did you hear about x education', 'what is your current occupation',
       'what matters most to you in choosing a course', 'search', 'magazine',
       'newspaper article', 'x education forums', 'newspaper',
       'digital advertisement', 'through recommendations',
       'receive more updates about our courses', 'tags', 'lead quality',
       'update me on supply chain content', 'get updates on dm content',
       'lead profile', 'city', 'asymmetrique activity index',
       'asymmetrique profile index',
       'i agree to pay the amount through cheque',
       'a free copy of mastering the interview', 'last notable activity'],
      dtype='object')

In [52]:
for col in cat_cols_new:
    if new_df[col].isnull().sum()>0:
        new_df[col].fillna('missing', inplace=True)

In [53]:
new_df[cat_cols_new]=new_df[cat_cols_new].apply(lambda col:col.str.lower())
new_df[cat_cols_new]=new_df[cat_cols_new].apply(lambda col:col.str.strip())

In [54]:
num_cols_new=new_df.select_dtypes(include=['int64','float64'])
num_cols_new.columns

Index(['lead number', 'converted', 'totalvisits',
       'total time spent on website', 'page views per visit',
       'asymmetrique activity score', 'asymmetrique profile score'],
      dtype='object')

In [55]:
for col in num_cols_new:
    if new_df[col].isnull().sum()>0:
        new_df.fillna(df[col].median(), inplace=True)

In [56]:
new_df.isnull().sum()

,0
prospect id,0
lead number,0
lead origin,0
lead source,0
do not email,0
do not call,0
converted,0
totalvisits,0
total time spent on website,0
page views per visit,0


In [57]:
selected_features = joblib.load("chi_selected_features.pkl")
new_df_encoded = pd.get_dummies(new_df, drop_first=True)
new_df_encoded = new_df_encoded.reindex(
    columns=selected_features,
    fill_value=0
)

In [58]:
predictions = {}
for name, model in models.items():
    predictions[name] = model.predict(new_df_encoded)

In [59]:
results_df = new_df.copy()
for name, preds in predictions.items():
    results_df[f"{name}_prediction"] = preds

In [60]:
results_df.head()

,prospect id,lead number,lead origin,lead source,do not email,do not call,converted,totalvisits,total time spent on website,page views per visit,...,last notable activity,Logistic Regression_prediction,Decision Tree_prediction,Random Forest_prediction,GradientBoosting_prediction,SVM_prediction,CatBoosting_prediction,XGBoostingClassifier_prediction,LightGBM_prediction,AdaBoostClassifier_prediction
0,8b94b8c4-7107-4c12-8a66-531d0ed974c4,615582.0987,landing page submission,google,no,no,0.903701,8.075989,251.953922,2.638773,...,email opened,1,1,1,1,0,1,1,1,1
1,a4c0a4e4-c5d8-4e48-8a4f-030611dde534,588939.0819,landing page submission,direct traffic,yes,no,0.057036,2.110425,928.953347,2.013832,...,modified,0,0,0,0,1,0,0,0,0
2,889c23c8-84d0-4936-97ad-7b438dc2e2d7,621241.8226,landing page submission,direct traffic,yes,no,-0.048638,1.123194,2.014230,1.096946,...,modified,0,0,0,0,0,0,0,0,0
3,85dfdbea-fb6c-4428-a59c-59155bedea4a,589802.8893,landing page submission,direct traffic,no,no,-0.024684,2.035459,323.092657,2.099535,...,email opened,0,0,0,0,0,0,0,0,0
4,cbc292d8-752f-47ea-92c6-07d4247638d5,651441.0348,landing page submission,google,no,no,0.025625,3.135290,201.014615,3.095191,...,email opened,0,0,0,0,0,0,0,0,0
